# Lab 1: Create SAP Sales Order Agent

Welcome to the first lab of the SAP Sales Order Agent workshop! In this lab, you'll create a basic SAP sales order management agent using Strands Agents and Amazon Bedrock.

## 🎯 Learning Objectives

By the end of this lab, you will:
- Understand the basics of Strands Agents
- Create a SAP sales order agent with core capabilities
- Test the agent with mock SAP data
- Prepare for integration with AgentCore services

## ⏱️ Estimated Time: 30 minutes

In [1]:
# Import required libraries
import sys
import os
from utils import (
    print_header, print_success, print_error, print_info, print_warning,
    check_aws_credentials, check_bedrock_access, create_mock_order_data,
    display_architecture_progress, workshop_progress
)

# Display lab header
print_header("Lab 1: Create SAP Sales Order Agent")
display_architecture_progress(1)

🚀 Lab 1: Create SAP Sales Order Agent
----------------------------------------
📋 Current Architecture - Lab 1
----------------------------------------

Lab 1: Basic Agent
┌─────────────────┐
│   SAP Agent     │
│   (Mock Data)   │
└─────────────────┘
        


In [2]:
# Check prerequisites
print_header("Checking Prerequisites", level=2)

# Check AWS credentials
aws_ok = check_aws_credentials()

# Check Bedrock access
bedrock_ok = check_bedrock_access()

if aws_ok and bedrock_ok:
    print_success("All prerequisites met! Ready to proceed.")
else:
    print_error("Please resolve the issues above before continuing.")

----------------------------------------
📋 Checking Prerequisites
----------------------------------------
✅ AWS credentials configured for account: 953841955037
✅ Model available: anthropic.claude-3-5-sonnet-20241022-v2:0
✅ Model available: anthropic.claude-3-5-haiku-20241022-v1:0
✅ All prerequisites met! Ready to proceed.


In [3]:
# Create mock SAP data
print_header("Creating Mock SAP Data", level=2)

mock_orders = create_mock_order_data()

print_info(f"Created {len(mock_orders)} mock sales orders:")
for order in mock_orders:
    status_icon = "🚫" if order['has_delivery_block'] else "✅"
    print(f"  {status_icon} {order['order_id']} - {order['customer_name']} - {order['order_value']} {order['currency']}")

print_success("Mock data created successfully!")

----------------------------------------
📋 Creating Mock SAP Data
----------------------------------------
ℹ️  Created 3 mock sales orders:
  🚫 SO001234 - ACME Corporation - 15000.0 USD
  🚫 SO001235 - TechCorp Ltd - 8500.0 USD
  ✅ SO001236 - Global Manufacturing Inc - 25000.0 USD
✅ Mock data created successfully!


In [14]:
from strands import Agent, tool
from strands.models import BedrockModel

# Define SAP-focused system prompt
SAP_SYSTEM_PROMPT = """You are a SAP Sales Order Agent with comprehensive order management capabilities. You can:

1. List sales orders with delivery blocks
2. Show detailed information for specific orders  
3. Remove delivery blocks from orders
4. Send email notifications
5. Provide troubleshooting guidance

When handling SAP orders:
- Always provide clear, professional responses
- Include relevant details like customer, value, and block reasons
- Format information in a user-friendly way
- Handle errors appropriately
- Explain SAP processes in business terms

You have access to tools for managing SAP sales orders. Use them to provide accurate, real-time information."""

@tool
def list_blocked_orders() -> str:
    """List all sales orders that have delivery blocks."""
    # This will be populated with actual order data
    return "Tool function - will be overridden in class"

@tool  
def get_order_details(order_id: str) -> str:
    """Get detailed information about a specific sales order."""
    return f"Tool function for {order_id} - will be overridden in class"

@tool
def remove_delivery_block(order_id: str, reason: str = "Manual removal") -> str:
    """Remove delivery block from a sales order."""
    return f"Tool function for {order_id} - will be overridden in class"

class SAPSalesOrderAgent:
    """SAP Sales Order Agent using proper Strands Agent API."""
    
    def __init__(self, mock_data: List[Dict[str, Any]]):
        """Initialize the agent with mock SAP data."""
        self.orders = {order['order_id']: order for order in mock_data}
        
        # Create tools that have access to self.orders
        @tool
        def list_blocked_orders() -> str:
            """List all sales orders that have delivery blocks."""
            blocked_orders = [order for order in self.orders.values() if order['has_delivery_block']]
            
            if not blocked_orders:
                return "No sales orders with delivery blocks found."
            
            result = f"Found {len(blocked_orders)} sales orders with delivery blocks:\n\n"
            
            for i, order in enumerate(blocked_orders, 1):
                block_info = order.get('delivery_block', {})
                result += f"{i}. **Order {order['order_id']}**\n"
                result += f"   - Customer: {order['customer_name']}\n"
                result += f"   - Value: {order['currency']} {order['order_value']:,.2f}\n"
                result += f"   - Block Reason: {block_info.get('reason', 'Unknown')}\n"
                result += f"   - Blocked Since: {block_info.get('blocked_date', 'Unknown')}\n\n"
            
            return result
        
        @tool
        def get_order_details(order_id: str) -> str:
            """Get detailed information about a specific sales order."""
            # Find the order
            order = None
            for oid, order_data in self.orders.items():
                if oid == order_id or oid.replace('SO', '') == order_id.replace('SO', ''):
                    order = order_data
                    break
            
            if not order:
                return f"Sales order {order_id} not found. Available orders: {', '.join(self.orders.keys())}"
            
            result = f"**Order Details for {order['order_id']}**\n\n"
            result += f"📦 **Order Information:**\n"
            result += f"- Order ID: {order['order_id']}\n"
            result += f"- Customer: {order['customer_name']} ({order['customer_number']})\n"
            result += f"- Order Date: {order['order_date']}\n"
            result += f"- Order Value: {order['currency']} {order['order_value']:,.2f}\n"
            result += f"- Status: {order['status']}\n"
            result += f"- Material: {order['material_description']}\n\n"
            
            if order['has_delivery_block']:
                block_info = order['delivery_block']
                result += f"🚫 **Delivery Block:**\n"
                result += f"- Block Reason: {block_info['reason']}\n"
                result += f"- Blocked Since: {block_info['blocked_date']}\n"
                result += f"- Blocked By: {block_info['blocked_by']}\n"
            else:
                result += f"✅ **No delivery blocks**\n"
            
            return result
        
        @tool
        def remove_delivery_block(order_id: str, reason: str = "Manual removal") -> str:
            """Remove delivery block from a sales order."""
            # Find and update the order
            for oid, order_data in self.orders.items():
                if oid == order_id or oid.replace('SO', '') == order_id.replace('SO', ''):
                    if order_data['has_delivery_block']:
                        order_data['has_delivery_block'] = False
                        order_data['delivery_block'] = None
                        return f"✅ **Delivery block removed from {order_data['order_id']}!**\n\nReason: {reason}\nThe order is now released for delivery processing."
                    else:
                        return f"Order {order_data['order_id']} does not have any delivery blocks."
            
            return f"Sales order {order_id} not found."
        
        # Create Bedrock model
        bedrock_model = BedrockModel(
            model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
            temperature=0.3,
        )
        
        # Create agent with system prompt, tools, and model
        self.agent = Agent(
            system_prompt=SAP_SYSTEM_PROMPT,
            tools=[list_blocked_orders, get_order_details, remove_delivery_block],
            model=bedrock_model,
        )
    
    def process_message(self, message: str) -> str:
        """Process a user message and return the agent's response."""
        try:
            response = self.agent(message)
            return response.message
        except Exception as e:
            return f"Error processing message: {str(e)}"

print_success("SAP Sales Order Agent class created successfully with proper Strands API!")


✅ SAP Sales Order Agent class created successfully with proper Strands API!


In [15]:
# Initialize the agent with mock data
print_header("Initializing SAP Agent", level=2)

sap_agent = SAPSalesOrderAgent(mock_orders)
print_success("SAP Sales Order Agent initialized successfully!")

print_info("Agent capabilities:")
print("  • List sales orders with delivery blocks")
print("  • Show detailed order information")
print("  • Remove delivery blocks from orders")
print("  • Send email notifications")
print("  • Provide troubleshooting guidance")

----------------------------------------
📋 Initializing SAP Agent
----------------------------------------
✅ SAP Sales Order Agent initialized successfully!
ℹ️  Agent capabilities:
  • List sales orders with delivery blocks
  • Show detailed order information
  • Remove delivery blocks from orders
  • Send email notifications
  • Provide troubleshooting guidance


In [16]:
# Test the agent
print_header("Testing SAP Agent", level=2)

# Test 1: List blocked orders
query1 = "Show me all sales orders with delivery blocks"
print(f"🧑‍💻 User: {query1}")
print("\n🤖 Agent:")
response1 = sap_agent.process_message(query1)
print(response1)

----------------------------------------
📋 Testing SAP Agent
----------------------------------------
🧑‍💻 User: Show me all sales orders with delivery blocks

🤖 Agent:
I'll retrieve the list of sales orders that currently have delivery blocks for you.
Tool #1: list_blocked_orders
I've retrieved the current sales orders with delivery blocks. There are 2 orders currently blocked:

1. Order SO001234 for ACME Corporation:
   - Total Value: $15,000.00
   - Blocked due to exceeding credit limit
   - Block placed on January 15, 2024

2. Order SO001235 for TechCorp Ltd:
   - Total Value: $8,500.00
   - Blocked due to incomplete documentation
   - Block placed on January 16, 2024

Would you like me to provide more detailed information about either of these orders or help you resolve the delivery blocks?{'role': 'assistant', 'content': [{'text': "I've retrieved the current sales orders with delivery blocks. There are 2 orders currently blocked:\n\n1. Order SO001234 for ACME Corporation:\n   - To

## 🎉 Lab 1 Complete!

You've successfully created a basic SAP Sales Order Agent! The key fix was initializing the Strands `Agent()` without parameters, as it doesn't accept `name` or `instructions` in the constructor.

Ready for Lab 2? **[Continue to Lab 2 →](lab-02-add-memory.ipynb)**